# Clausius Rankine Cycle


## Introduction

The clausius rankine cycle serves as a comparative process for the steam power plant in its simplest constellation with steam turbine, condenser, feed water pump and steam generator shown in figure {numref}`rankine-cycle-dissipative-flowsheet`. This is based on the heat-power process occurring in a steam engine and is thus a clockwise rotationg cycle.

```{figure} /figures/rankine_cycle_dissipative.svg
---
name: rankine-cycle-dissipative-flowsheet
---
Topology of the clausius rankine cycle.
```

The live steam provided in the steam generator enters the turbine at high pressure and temperature, where it is expanded to a lower pressure level. The turbine releases power that can be used to drive a generator. The exhaust steam leaving the turbine is completely condensed out and leaves the condenser as saturated water. After that the feed water pump has to bring this condensate to a pressure level so that the steam generator can be supplied with feed water and the cycle can start again.

The heat supplied to the process comes from the steam generator. In contrast, the power output of the turbine differentiated by the power input of the feedwater pump, is to be defined as power output of the process. The condensation of the refrigerant is to be regarded as heat output. Consequently, the thermal efficiency is displayed in Eq. {eq}`thermal-efficiency`.

```{math}
    :label: thermal-efficiency
    \eta_{th} = \frac{W_{out}}{Q_{in}}
```

## Excercise 1 

Build the clausius rankine cycle in two variants:

1. A cycle with a dissipative condenser (system boundaries are around the hot side of the heat exchanger)
2. A condenser with the cold side, e.g. a river water cooling cycle, modelled (system boundaries are in and out flowing water)

Make sure the cycles are the same by checking their respective thermal efficiencies. Perform and compare the exergy analysis of the two variants. Interprete the results and the implication on the condensers efficiency.

Parametrize the model according to the description as shown in Table {numref}`clausius-rankine-param`.

```{list-table} Parametrization of clausius rankine clcye
:header-rows: 1
:name: clausius-rankine-param

* - Description
  - Parameter
  - Value
  - Unit
* - Network parameter
  - Massflow
  - 10
  - kg / s
* - Turbine
  - Isentropic efficiency
  - 90
  - \%
* - 
  - Live steam temperature
  - 600
  - °C
* - 
  - Live steam pressure
  - 150
  - bar
* - 
  - exhaust steam pressure
  - 0.1
  - bar
* - Feed water pump
  - Isentropic efficiency
  - 75
  - \%
* - Steam generator
  - Pressure losses
  - 10
  - \%
* - Condenser
  - Pressure losses
  - 0
  - \%
* - Cooling Water Source
  - Incoming Temperature
  - 20
  - °C
* - 
  - Temperature increase
  - 10
  - K
* - 
  - System pressure
  - 1.2
  - bar
```


### Proposed solution 1.1

**First step:** Build clausius rankine cycle with condenser using `HeatExchangerSimple` component

In [1]:
from tespy.networks import Network
from tespy.connections import Connection, Bus
from tespy.components import CycleCloser, Pump, HeatExchangerSimple, Turbine

# Network
nw = Network(fluids=['water'])
nw.set_attr(T_unit='C', p_unit='bar', h_unit='kJ / kg')

# Components
cycle_closer = CycleCloser('Cycle Closer')
turbine = Turbine('Steam Turbine')
condenser = HeatExchangerSimple('Condenser')
fw_pump = Pump('Feed Water Pump')
steam_generator = HeatExchangerSimple('Steam Generator')

# Connections
steamgen2cc = Connection(steam_generator, 'out1', cycle_closer, 'in1', label='0')
cc2turb = Connection(cycle_closer, 'out1', turbine, 'in1', label='1')
turb2cond = Connection(turbine, 'out1', condenser, 'in1', label='2')
cond2pump = Connection(condenser, 'out1', fw_pump, 'in1', label='3')
pump2steamgen = Connection(fw_pump, 'out1', steam_generator, 'in1', label='4')

nw.add_conns(steamgen2cc, cc2turb, turb2cond, cond2pump, pump2steamgen)

# Component Parametrization
turbine.set_attr(eta_s=0.9)
condenser.set_attr(pr=1)
fw_pump.set_attr(eta_s=0.75)
steam_generator.set_attr(pr=0.9)

# Connection Parametrization
cc2turb.set_attr(T=600, p=150, m=10, fluid={'water': 1})
turb2cond.set_attr(p=0.1)
cond2pump.set_attr(x=0)

# Busses
heat_in = Bus('Heat Input')
heat_in.add_comps({'comp': steam_generator, 'base': 'bus'})

power_out = Bus('Power Output')
power_out.add_comps(
    {'comp': turbine, 'char': 0.96, 'base': 'component'},
    {'comp': fw_pump, 'char': 0.96, 'base': 'bus'}
    )

heat_out = Bus('Heat Output')
heat_out.add_comps({'comp': condenser, 'base': 'component'})

nw.add_busses(heat_in, power_out, heat_out)

**Second step:** Solve model with variant 1 and perform exergy analysis

In [2]:
from tespy.tools import ExergyAnalysis

# Solve model
nw.set_attr(iterinfo=False)
nw.solve(mode='design')
eta_th = abs(power_out.P.val)/heat_in.P.val
print(f'eta_th = {eta_th:.3f}')

ean_var1 = ExergyAnalysis(network=nw, E_F=[heat_in], E_P=[power_out], E_L=[heat_out])
ean_var1.analyse(pamb=1.013, Tamb=20)
ean_var1.print_results(connections=False)

eta_th = 0.370
##### RESULTS: Component exergy analysis #####
+-----------------+-------------+-------------+-------------+-------------+-------------+-------------+
|                 |         E_F |         E_P |         E_D |     epsilon |        y_Dk |       y*_Dk |
|-----------------+-------------+-------------+-------------+-------------+-------------+-------------|
| Condenser       |   1.675e+06 | nan         |   1.675e+06 | nan         |   1.042e-01 |   4.615e-01 |
| Steam Turbine   |   1.456e+07 |   1.321e+07 |   1.349e+06 |   9.073e-01 |   8.390e-02 |   3.717e-01 |
| Feed Water Pump |   2.236e+05 |   1.724e+05 |   5.118e+04 |   7.711e-01 |   3.183e-03 |   1.410e-02 |
| Steam Generator |   1.608e+07 |   1.606e+07 |   1.658e+04 |   9.990e-01 |   1.031e-03 |   4.568e-03 |
| Cycle Closer    | nan         | nan         | nan         | nan         | nan         | nan         |
+-----------------+-------------+-------------+-------------+-------------+-------------+-------------+
##

**Third step:** Replace `HeatExchangerSimple` with `Condenser` component

In [3]:
from tespy.components import Condenser, Source, Sink
    
# Remove old connections
nw.del_conns(turb2cond, cond2pump)

# Replace condenser
condenser = Condenser('Condenser')
cw_source = Source('Cooling Water Source')
cw_sink = Sink('Cooling Water Sink')

# Reconnect condenser
turb2cond = Connection(turbine, 'out1', condenser, 'in1', label='2')
cond2pump = Connection(condenser, 'out1', fw_pump, 'in1', label='3')
cw_source2cond = Connection(cw_source, 'out1', condenser, 'in2', label='11')
cond2cw_sink = Connection(condenser, 'out2', cw_sink, 'in1', label='12')

nw.add_conns(turb2cond, cond2pump, cw_source2cond, cond2cw_sink)

# Parametrize condenser and cooling cycle
turb2cond.set_attr(p=0.1)
condenser.set_attr(pr1=1, pr2=1)
cw_source2cond.set_attr(T=20, p=1.2, fluid={'water': 1})
cond2cw_sink.set_attr(T=30)

# Reset heat output bus
nw.del_busses(heat_out)

heat_out = Bus('Heat Output')
heat_out.add_comps(
    {'comp': cw_source, 'base': 'bus'},
    {'comp': cw_sink, 'base': 'component'}
    )

nw.add_busses(heat_out)

**Forth step:** Solve model with variant 2 and perform exergy analysis

In [4]:
# Solve model
nw.solve(mode='design')
eta_th = abs(power_out.P.val)/heat_in.P.val
print(f'eta_th = {eta_th:.3f}')

ean_var2 = ExergyAnalysis(network=nw, E_F=[heat_in], E_P=[power_out], E_L=[heat_out])
ean_var2.analyse(pamb=1.013, Tamb=20)
ean_var2.print_results(groups=False, connections=False, aggregation=False)

eta_th = 0.370
##### RESULTS: Component exergy analysis #####
+----------------------+-------------+-------------+-------------+-------------+-------------+-------------+
|                      |         E_F |         E_P |         E_D |     epsilon |        y_Dk |       y*_Dk |
|----------------------+-------------+-------------+-------------+-------------+-------------+-------------|
| Steam Turbine        |   1.456e+07 |   1.321e+07 |   1.349e+06 |   9.073e-01 |   8.390e-02 |   4.108e-01 |
| Condenser            |   1.675e+06 |   3.452e+05 |   1.330e+06 |   2.061e-01 |   8.270e-02 |   4.049e-01 |
| Feed Water Pump      |   2.236e+05 |   1.724e+05 |   5.118e+04 |   7.711e-01 |   3.183e-03 |   1.558e-02 |
| Steam Generator      |   1.608e+07 |   1.606e+07 |   1.658e+04 |   9.990e-01 |   1.031e-03 |   5.048e-03 |
| Cycle Closer         | nan         | nan         | nan         | nan         | nan         | nan         |
| Cooling Water Sink   | nan         | nan         | nan         |

```{admonition} Explanation
:class: dropdown

First variation makes Condenser look very inefficient, because all exergy is destroyed. Second variation highlights real losses of the condenser and dissipates heat through cooling water.
```

```{attention}

From this point onwards the second variant of the clausius rankine cycle containg the implementation of the condensers cooling cycle is used. 
```

## Efficiency improvements and exergy influence

As you should already know, thermal efficiency depends on component parameters. Especially the variations of live steam parameters have a positive effect on thermal efficiency. Thus, both high temperatures and high pressures improve the thermal efficiency. However, if the live steam pressure is increased without simultaneously increasing the live steam temperature, the last stages of the turbine may be damaged by droplet impingement. But how do these key parameter affect exergetic efficiency? Are there other factors that influence it as well?

## Excercise 2

1. Analyze the influence of key parameters on the exergy analysis, such as live steam pressure and temperature.
2. What impact does the ambient state have on the results of the exergy analysis.

### Proposed solution 2.1

In [ ]:
import matplotlib.pyplot as plt

T_ls_range = [*range(500, 750, 50)]
p_ls_range = [*range(100, 225, 25)]

fig, ax = plt.subplots(figsize=(8, 6))

for p_ls in p_ls_range:
    epsilon_tot = list()
    for T_ls in T_ls_range:
        cc2turb.set_attr(T=T_ls, p=p_ls)

        nw.solve('design')
        ean = ExergyAnalysis(network=nw, E_F=[heat_in], E_P=[power_out], E_L=[heat_out])
        ean.analyse(pamb=1.013, Tamb=20)
        epsilon_tot.append(ean.network_data['epsilon'])
    ax.plot(T_ls_range, epsilon_tot, label=f'{p_ls} bar')

ax.set_xlabel('Live steam temperature in °C')
ax.set_ylabel('Exergetic efficiency $\\epsilon$')
ax.legend(title='$p_{livesteam}$:') #, alignment='left')
ax.grid()
ax.set_axisbelow(True)

### Proposed solution 2.2

In [ ]:
# Reset live steam parameters
cc2turb.set_attr(T=600, p=150)
nw.solve('design')
ean = ExergyAnalysis(network=nw, E_F=[heat_in], E_P=[power_out], E_L=[heat_out])

T_amb_range = [*range(5, 40, 5)]

E_Fs = list()
E_Ps = list()
epsilons = list()
for T_amb in T_amb_range:
    ean.analyse(pamb=1.013, Tamb=T_amb)
    E_Fs.append(ean.network_data['E_F'] * 1e-6)
    E_Ps.append(ean.network_data['E_P'] * 1e-6)
    epsilons.append(ean.network_data['epsilon'])

fig, ax1 = plt.subplots(figsize=(8, 6))

ax1.plot(T_amb_range, E_Fs, color='#00395B',  label='$\\dot{E}_{F}$')
ax1.plot(T_amb_range, E_Ps, color='#B54036', label='$\\dot{E}_{P}$')

ax2 = ax1.twinx()

ax2.plot(T_amb_range, epsilons, color='#EC6707', ls=':', label='$\\epsilon$')

ax1.set_xlabel('Ambient temperature in °C')
ax1.set_ylabel('Total Exergies in MW')
ax1.legend()
ax1.grid()
ax1.set_axisbelow(True)

ax2.set_ylabel('Exergetic efficiency')
ax2.legend()

```{admonition} Explanation
:class: dropdown

Hier könnte ihre Erklärung stehen
```

## Process optimization

In addition to the approach of increasing the efficiency via parameter variation, there is also the approach of structural optimization. This involves modifying the circuitry of the process with the aim of increasing efficiency, exergy efficiency or other improvement.

The regenerative feed water preheating is an often used structural optimization. Figure {numref}`rankine-cycle-preheating-flowsheet` shows the structure of such a process. Most of the live steam entering the turbine is expanded to the condenser, and from there it reaches the feedwater pump. However, the cold feedwater is now preheated before it enters the boiler. This is done by condensing steam taken from the turbine at a suitable pressure level in a feed water preheater. In the present case, the resulting extraction steam condensate is returned to the condenser of the steam power process. This requires throttling to balance the pressures.

```{figure} /figures/rankine_cycle_preheating.svg
---
name: rankine-cycle-preheating-flowsheet
---
Topology of the clausius rankine cycle with regenerative feed water preheating.
```

A second way of increasing the efficiency of a clausius rankine process is superheating. This additionally influences the operating range of the turbines used, which on the one hand means a technical advantage, but on the other hand can also open up the scope for further parameter optimization. Figure {numref}`rankine-cycle-superheating-flowsheet` shows the structure of a Rankine Cycle with superheating. The live steam, coming from the steam generator, enters the turbine, which now consists of a high-pressure and a low-pressure section. After initial expansion has taken place in the high-pressure section, all the steam is removed from the turbine and fed to an intermediate superheater. The steam then flows into the low-pressure section of the turbine, from where the process continues as known from the simple process.

```{figure} /figures/rankine_cycle_superheating.svg
---
name: rankine-cycle-superheating-flowsheet
---
Topology of the clausius rankine cycle with intermediate superheating of steam.
```

## Excercise 3

1. Implement the two topological improvements:
    - Regenerative feed water preheating (see {numref}`rankine-cycle-preheating-flowsheet`)
    - Intermediate superheating (see {numref}`rankine-cycle-superheating-flowsheet`)
2. Explain the improvement of the cycles efficiency with the exergy analysis by investigating the exergy destruction of each component

### Proposed solution 3.1

**First step:** Build clausius rankine cycle with regenerative feed water preheating

In [ ]:
from tespy.connections import Ref
from tespy.components import Splitter, Merge, HeatExchanger, Valve
import numpy as np

# Network
nw_pre = Network(fluids=['water'])
nw_pre.set_attr(T_unit='C', p_unit='bar', h_unit='kJ / kg')

# Components
cycle_closer = CycleCloser('Cycle Closer')
hp_turbine = Turbine('High Pressure Steam Turbine')
splitter = Splitter('Splitter')
lp_turbine = Turbine('Low Pressure Steam Turbine')
merge = Merge('Merge')

condenser = Condenser('Condenser')
cw_source = Source('Cooling Water Source')
cw_sink = Sink('Cooling Water Sink')
fw_pump = Pump('Feed Water Pump')
steam_generator = HeatExchangerSimple('Steam Generator')

valve = Valve('Expansion Valve')
preheater = HeatExchanger('Preheater')

# Connections
steamgen2cc = Connection(steam_generator, 'out1', cycle_closer, 'in1', label='0')
cc2hp_turb = Connection(cycle_closer, 'out1', hp_turbine, 'in1', label='1')
hp_turb2split = Connection(hp_turbine, 'out1', splitter, 'in1', label='2')
split2lp_turb = Connection(splitter, 'out1', lp_turbine, 'in1', label='3')
lp_turb2merge = Connection(lp_turbine, 'out1', merge, 'in1', label='4')

nw_pre.add_conns(steamgen2cc, cc2hp_turb, hp_turb2split, split2lp_turb, lp_turb2merge)

merg2cond = Connection(merge, 'out1', condenser, 'in1', label='5')
cond2pump = Connection(condenser, 'out1', fw_pump, 'in1', label='6')
pump2preheat = Connection(fw_pump, 'out1', preheater, 'in2', label='7')
preheat2steamgen = Connection(preheater, 'out2', steam_generator, 'in1', label='8')
cw_source2cond = Connection(cw_source, 'out1', condenser, 'in2', label='21')
cond2cw_sink = Connection(condenser, 'out2', cw_sink, 'in1', label='22')

nw_pre.add_conns(merg2cond, cond2pump, pump2preheat, preheat2steamgen, cw_source2cond, cond2cw_sink)

split2preheat = Connection(splitter, 'out2', preheater, 'in1', label='11')
preheat2valve = Connection(preheater, 'out1', valve, 'in1', label='12')
valve2merge = Connection(valve, 'out1', merge, 'in2', label='13')

nw_pre.add_conns(split2preheat, preheat2valve, valve2merge)

# Component Parametrization
hp_turbine.set_attr(eta_s=0.9)
lp_turbine.set_attr(eta_s=0.9)
condenser.set_attr(pr1=1, pr2=1)
fw_pump.set_attr(eta_s=0.75)
preheater.set_attr(pr1=1, pr2=1, ttd_l=5)
steam_generator.set_attr(pr=0.9)

# Connection Parametrization
cc2hp_turb.set_attr(T=600, p=150, m=10, fluid={'water': 1})
split2lp_turb.set_attr(p=np.sqrt(150*0.1), m=Ref(cc2hp_turb, 0.85, 0))
lp_turb2merge.set_attr(p=0.1)

cw_source2cond.set_attr(T=20, p=1.2, fluid={'water': 1})
cond2cw_sink.set_attr(T=30)

# Busses
heat_in = Bus('Heat Input')
heat_in.add_comps({'comp': steam_generator, 'base': 'bus'})

power_out = Bus('Power Output')
power_out.add_comps(
    {'comp': hp_turbine, 'char': 0.96, 'base': 'component'},
    {'comp': lp_turbine, 'char': 0.96, 'base': 'component'},
    {'comp': fw_pump, 'char': 0.96, 'base': 'bus'}
    )

heat_out = Bus('Heat Output')
heat_out.add_comps(
    {'comp': cw_source, 'base': 'bus'},
    {'comp': cw_sink, 'base': 'component'}
    )

nw_pre.add_busses(heat_in, power_out, heat_out)


**Second step:** Solve model with regenerative feed water preheating and perform exergy analysis

In [ ]:
# Solve model
nw_pre.set_attr(iterinfo=False)
nw_pre.solve(mode='design')
eta_th = abs(power_out.P.val)/heat_in.P.val
print(f'eta_th = {eta_th:.3f}')

ean_pre = ExergyAnalysis(network=nw_pre, E_F=[heat_in], E_P=[power_out], E_L=[heat_out])
ean_pre.analyse(pamb=1.013, Tamb=20)
ean_pre.print_results(groups=False, connections=False, aggregation=False)

**Third step:** Build clausius rankine cycle with intermediate superheating of steam

In [ ]:
nw_super = Network(fluids=['water'])
nw_super.set_attr(T_unit='C', p_unit='bar', h_unit='kJ / kg')

# Components
cycle_closer = CycleCloser('Cycle Closer')
hp_turbine = Turbine('High Pressure Steam Turbine')
steam_superheater = HeatExchangerSimple('Steam Superheater')
lp_turbine = Turbine('Low Pressure Steam Turbine')
condenser = Condenser('Condenser')
fw_pump = Pump('Feed Water Pump')
steam_generator = HeatExchangerSimple('Steam Generator')

cw_source = Source('Cooling Water Source')
cw_sink = Sink('Cooling Water Sink')


# Connections
steamgen2cc = Connection(steam_generator, 'out1', cycle_closer, 'in1', label='0')
cc2hp_turb = Connection(cycle_closer, 'out1', hp_turbine, 'in1', label='1')
hp_turb2super = Connection(hp_turbine, 'out1', steam_superheater, 'in1', label='2')
super2lp_turb = Connection(steam_superheater, 'out1', lp_turbine, 'in1', label='3')
lp_turb2cond = Connection(lp_turbine, 'out1', condenser, 'in1', label='4')
cond2pump = Connection(condenser, 'out1', fw_pump, 'in1', label='5')
pump2steamgen = Connection(fw_pump, 'out1', steam_generator, 'in1', label='6')

cw_source2cond = Connection(cw_source, 'out1', condenser, 'in2', label='11')
cond2cw_sink = Connection(condenser, 'out2', cw_sink, 'in1', label='12')

nw_super.add_conns(
    steamgen2cc, cc2hp_turb, hp_turb2super, super2lp_turb, lp_turb2cond,
    cond2pump, pump2steamgen, cw_source2cond, cond2cw_sink
    )

# Component Parametrization
hp_turbine.set_attr(eta_s=0.9)
lp_turbine.set_attr(eta_s=0.9)
condenser.set_attr(pr1=1, pr2=1)
fw_pump.set_attr(eta_s=0.75)
steam_generator.set_attr(pr=0.9)
steam_superheater.set_attr(pr=0.9)

# Connection Parametrization
cc2hp_turb.set_attr(T=600, p=150, m=10, fluid={'water': 1})
hp_turb2super.set_attr(p=np.sqrt(150*0.1))
super2lp_turb.set_attr(T=600)
lp_turb2cond.set_attr(p=0.1)

cw_source2cond.set_attr(T=20, p=1.2, fluid={'water': 1})
cond2cw_sink.set_attr(T=30)

# Busses
heat_in = Bus('Heat Input')
heat_in.add_comps(
    {'comp': steam_generator, 'base': 'bus'},
    {'comp': steam_superheater, 'base': 'bus'}
    )

power_out = Bus('Power Output')
power_out.add_comps(
    {'comp': hp_turbine, 'char': 0.96, 'base': 'component'},
    {'comp': lp_turbine, 'char': 0.96, 'base': 'component'},
    {'comp': fw_pump, 'char': 0.96, 'base': 'bus'}
    )

heat_out = Bus('Heat Output')
heat_out.add_comps(
    {'comp': cw_source, 'base': 'bus'},
    {'comp': cw_sink, 'base': 'component'}
    )

nw_super.add_busses(heat_in, power_out, heat_out)

**Forth step:** Solve model with intermediate superheater and perform exergy analysis

In [ ]:
# Solve model
nw_super.set_attr(iterinfo=False)
nw_super.solve(mode='design')
eta_th = abs(power_out.P.val)/heat_in.P.val
print(f'eta_th = {eta_th:.3f}')

ean_super = ExergyAnalysis(network=nw_super, E_F=[heat_in], E_P=[power_out], E_L=[heat_out])
ean_super.analyse(pamb=1.013, Tamb=20)
ean_super.print_results(groups=False, connections=False, aggregation=False)

**Fifth step:** Compare the two topological improvements

In [ ]:
# Compare the two topological improvements

```{admonition} Explanation
:class: dropdown

Hier könnte ihre Erklärung stehen
```

## Lessons Learned

- Lessons Learned 1
- Lessons Learned 2
- Lessons Learned 3